In [ ]:
%pip install category_encoders

# Categoical Variables

- BuildingType
- PrimaryPropertyType
- PropertyName
- Address
- City
- State
- ZipCode
- TaxParcelIdentificationNumber
- CouncilDistrictCode
- Neighborhood
- DefaultData
- ComplianceStatus
- Outlier

In [9]:
import pandas as pd

pd.set_option('display.max_columns', None)

df = pd.read_csv("2016_Building_Energy_Benchmarking.csv")

df

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7.226362e+06,7.456910e+06,2003882.00,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,61.0,94.800003,97.900002,176.100006,179.399994,8.387933e+06,8.664479e+06,0.00,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,Hotel,756493.0,NaN,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,7.258702e+07,7.393711e+07,21566554.00,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,61320,0,61320,Hotel,Hotel,61320.0,NaN,NaN,NaN,NaN,NaN,56.0,110.800003,113.300003,216.199997,224.000000,6.794584e+06,6.946800e+06,2214446.25,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,175580,62000,113580,"Hotel, Parking, Swimming Pool",Hotel,123445.0,Parking,68009.0,Swimming Pool,0.0,NaN,75.0,114.800003,118.699997,211.399994,215.600006,1.417261e+07,1.465650e+07,0.00,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,False,NaN,Compliant,NaN,505.01,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,2,GREATER DUWAMISH,47.56722,-122.31154,1990,1.0,1,12294,0,12294,Office,Office,12294.0,NaN,NaN,NaN,NaN,NaN,46.0,69.099998,76.699997,161.699997,176.100006,8.497457e+05,9.430032e+05,0.00,1.536550e+05,5.242709e+05,3254.750244,3.254750e+05,True,NaN,Error - Correct Default Data,NaN,20.94,1.70
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,2,DOWNTOWN,47.59625,-122.32283,2004,1.0,1,16000,0,16000,Other - Recreation,Other - Recreation,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,59.400002,65.900002,114.199997,118.900002,9.502762e+05,1.053706e+06,0.00,1.162210e+05,3.965461e+05,5537.299805,5.537300e+05,False,NaN,Compliant,NaN,32.17,2.01
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,7,MAGNOLIA / QUEEN ANNE,47.63644,-122.35784,1974,1.0,1,13157,0,13157,"Fitness Center/Health Club/Gym, Other - Recrea...",Other - Recreation,7583.0,Fitness Center/Health Club/Gym,5574.0,Swimmin

In [10]:
# Function to drop missing and duplicate values
def clean_data(df):
    """
    Function to drop all columns with 100% missing values and duplicate rows.
    :param df: Input pandas DataFrame
    """
    # Drop columns with 100% missing values
    df.dropna(how="all", axis="columns", inplace=True)
    
    # Drop duplicate rows
    df.drop_duplicates(inplace=True)

clean_data(df)

In [16]:
def missing_data_info(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to calculate the percentage of missing data for each column and the indices of rows with missing data.

    Args:
    - df (pd.DataFrame): Input dataframe

    Returns:
    - pd.DataFrame: A dataframe with columns 'percentage_missing' and 'missing_rows'
    """

    # Percentage of missing data for each column
    missing_percentage = df.isnull().mean() * 100

    # Rows where data is missing for each column
    missing_rows = df.apply(lambda col: df[col.name][col.isnull()].index.tolist())

    # Combining the results
    result_df = pd.DataFrame({
        'percentage_missing': missing_percentage,
        'missing_rows': missing_rows
    })

    # Filter out columns with 0% missing data
    result_df = result_df[result_df['percentage_missing'] > 0]

    return result_df.round(2)

missing_data_info(df)

In [24]:
print(df["Outlier"].unique().tolist())
print(df["YearsENERGYSTARCertified"].unique().tolist())

[nan, 'High outlier', 'Low outlier']
[nan, '2016', '2014', '2012', '20172015', '2017201620152014201320122011201020092008', '201020092007', '201720102007', '20162015', '20152014201220112009', '201220092008', '201620152014201320112010200920072000', '201620092008', '201620142012', '20162015201420122008', '2016201520142013201220112010200920082007200620052004', '201520142013', '2014201320112010200920082007', '2016201520142013', '2017201620152014201320112008', '201720162015201420132008', '20162015201320122010', '2015201320122010', '20162011201020092008', '2007', '2010', '20172016201520092008', '2017201320092008', '2015', '2017', '2017201120102007', '201720162014201320122011201020092007', '201720152014201220102008', '20092005', '2017201520142013', '201620152012', '2012201020072006', '20112009', '2009', '201620152014201320092008', '201620152014201320112009', '2016201520142013201220112010', '201720162012', '20102008', '201620142012201120082007', '2017201520142013201120092008', '2016201520142013

In [27]:
def count_years(column: pd.Series) -> pd.Series:
    """
    Function to count the number of years (represented as 4-digit numbers) in each string in the given series.
    If a value is NaN, it returns 0.
    
    Args:
    - column (pd.Series): Input series with strings containing concatenated years
    
    Returns:
    - pd.Series: A series with counts of years for each string
    """
    return column.fillna('').astype(str).apply(lambda x: len([x[i:i+4] for i in range(0, len(x), 4)]))

df["YearsENERGYSTARCertified"] = count_years(df["YearsENERGYSTARCertified"])

In [28]:
missing_data_info(df)

,percentage_missing,missing_rows
ZipCode,0.47,"[3360, 3361, 3362, 3363, 3364, 3365, 3366, 336..."
NumberofBuildings,0.24,"[578, 758, 1768, 1792, 2321, 2474, 2480, 2670]"
ListOfAllPropertyUseTypes,0.27,"[578, 758, 1768, 1792, 2321, 2474, 2480, 2670,..."
LargestPropertyUseType,0.59,"[353, 578, 758, 1077, 1147, 1483, 1756, 1768, ..."
LargestPropertyUseTypeGFA,0.59,"[353, 578, 758, 1077, 1147, 1483, 1756, 1768, ..."
SecondLargestPropertyUseType,50.27,"[0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 19, 20, 2..."
SecondLargestPropertyUseTypeGFA,50.27,"[0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 19, 20, 2..."
ThirdLargestPropertyUseType,82.35,"[0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
ThirdLargestPropertyUseTypeGFA,82.35,"[0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
ENERGYSTARScore,24.97,"[5, 7, 15, 16, 18, 21, 24, 25, 26, 28, 31, 32,..."


# Transform Categorical Variables

In [7]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

# Assuming we have a dataframe called df
ohe = OneHotEncoder(drop='first', sparse_output=False)
target_enc = TargetEncoder()

# OneHotEncode low cardinality features
low_cardinality_features = ['BuildingType', 'City', 'State', 'CouncilDistrictCode', 'ComplianceStatus', 'Outlier']
encoded_low_cardinality = ohe.fit_transform(df[low_cardinality_features])

# TargetEncode high cardinality features (assuming the target variable is 'GHGEmissionsIntensity')
high_cardinality_features = ['PropertyName', 'Address']
encoded_high_cardinality = target_enc.fit_transform(df[high_cardinality_features], df['GHGEmissionsIntensity'])

# Dropping original columns and adding encoded ones
df.drop(columns=low_cardinality_features + high_cardinality_features, inplace=True)
df = pd.concat([df, pd.DataFrame(encoded_low_cardinality), pd.DataFrame(encoded_high_cardinality)], axis=1)


# Create New Variables from Existing Ones

In [8]:
df['AgeOfBuilding'] = 2023 - df['YearBuilt']  # Assuming current year is 2023

,OSEBuildingID,DataYear,PrimaryPropertyType,ZipCode,TaxParcelIdentificationNumber,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,TotalGHGEmissions,GHGEmissionsIntensity,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,PropertyName,Address
0,1,2016,Hotel,98101.0,0659000030,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7.226362e+06,7.456910e+06,2003882.00,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,False,NaN,249.98,2.83,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.391127,1.391127
1,2,2016,Hotel,98101.0,0659000220,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,61.0,94.800003,97.900002,176.100006,179.399994,8.387933e+06,8.664479e+06,0.00,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,False,NaN,295.86,2.86,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.395030,1.395030
2,3,2016,Hotel,98101.0,0659000475,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,Hotel,756493.0,NaN,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,7.258702e+07,7.393711e+07,21566554.00,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,False,NaN,2089.28,2.19,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.307857,1.307857
3,5,2016,Hotel,98101.0,0659000640,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,61320,0,61320,Hotel,Hotel,61320.0,NaN,NaN,NaN,NaN,NaN,56.0,110.800003,113.300003,216.199997,224.000000,6.794584e+06,6.946800e+06,2214446.25,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,False,NaN,286.43,4.67,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.630526,1.630526
4,8,2016,Hotel,98121.0,0659000970,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,175580,62000,113580,"Hotel, Parking, Swimming Pool",Hotel,123445.0,Parking,68009.0,Swimming Pool,0.0,NaN,75.0,114.800003,118.699997,211.399994,215.600006,1.417261e+07,1.465650e+07,0.00,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,False,NaN,505.01,2.88,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.397632,1.397632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Office,NaN,1624049080,GREATER DUWAMISH,47.56722,-122.31154,1990,1.0,1,12294,0,12294,Office,Office,12294.0,NaN,NaN,NaN,NaN,NaN,46.0,69.099998,76.699997,161.699997,176.100006,8.497457e+05,9.430032e+05,0.00,1.536550e+05,5.242709e+05,3254.750244,3.254750e+05,True,NaN,20.94,1.70,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.244104,1.244104
3372,50223,2016,Other,NaN,3558300000,DOWNTOWN,47.59625,-122.32283,2004,1.0,1,16000,0,16000,Other - Recreation,Other - Recreation,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,59.400002,65.900002,114.199997,118.900002,9.502762e+05,1.053706e+06,0.00,1.162210e+05,3.965461e+05,5537.299805,5.537300e+05,False,NaN,32.17,2.01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.284438,1.284438
3373,50224,2016,Other,NaN,1794501150,MAGNOLIA / QUEEN ANNE,47.63644,-122.35784,1974,1.0,1,13157,0,13157,"Fitness Center/Health Club/Gym, Other - Recrea...",Other - Recreation,7583.0,Fitness Center/Health Club/Gym,5574.0,S

# Mathematical Transformations

In [ ]:
import numpy as np

# Log transform
df['Log_SiteEnergyUse'] = np.log1p(df['SiteEnergyUse(kBtu)'])


# Normalize Variables 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_to_normalize = ['Latitude', 'Longitude']

df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])
